In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Model, GPT2Config, PreTrainedModel
from datasets import load_dataset, Dataset, DatasetDict

from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments

import math

In [2]:
class GPT2Trader(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        
        # use levine 2020 layer numbers
        n_layer = round((math.log(config.n_embd) - 5.039) / 5.55e-2)
        n_layer = max(1, n_layer)
        print(f'Using {n_layer} layers')
        config.n_layer = n_layer
        
        config.initializer_range = 1 / math.sqrt(config.n_embd)
        
        self.embed = nn.Linear(5, config.n_embd, bias = False)
        self.norm = nn.LayerNorm(config.n_embd)
        self.gpt = GPT2Model(config)
        self.trade = nn.Linear(config.n_embd, 60, bias = False)
        
        self.trade_sign = nn.Parameter(torch.Tensor([1, -1]), requires_grad = False)
        
    def forward(self, ohlcv, future):
        embed = self.norm(self.embed(ohlcv))
        hidden = self.gpt(inputs_embeds = embed).last_hidden_state
        
        soft_trade = self.trade(hidden)
        
        # sharpe information
        soft_trade = torch.tanh(soft_trade).reshape(-1)
        future = future.reshape(-1)
        soft_profit = soft_trade * future
        
        # the exp is so that loss is purely positive and minimizes toward 0 (also losses have more loss than profit)
        loss_ppl = (-soft_profit + future.abs()).mean()
        
        # penalty for big trades (to stop trading from happening with no profit)
        trade_penalty = soft_trade.abs().mean()
        
        loss = loss_ppl + .1 * trade_penalty # .1 means that a 100% position must make at least .1 of a std
        
        trades = soft_trade.abs().sum()
        print(f'sharpe {soft_profit.mean() / soft_profit.std():.4}, t-value {(soft_profit.mean() * trades**.5) / soft_profit.std():.4} '
              f'trade % {trades * 100 / len(soft_profit):,.4}, big trade % {(soft_trade.abs() > .8).sum() * 100 / len(soft_profit):.4}, loss {loss_ppl.item():.4}')
        
        return {"loss": loss}

In [3]:
class TraderTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.get("labels")
#         # forward pass
#         outputs = model(**inputs)
#         logits = outputs.get("logits")
#         # compute custom loss (suppose one has 3 labels with different weights)
#         loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
#         loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
#         return (loss, outputs) if return_outputs else loss

    def get_train_dataloader(self) -> DataLoader:
        """
        Returns the training :class:`~torch.utils.data.DataLoader`.

        Will use no sampler if :obj:`self.train_dataset` does not implement :obj:`__len__`, a random sampler (adapted
        to distributed training if necessary) otherwise.

        Subclass and override this method if you want to inject some custom behavior.
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        train_sampler = self._get_train_sampler()

        return DataLoader(
            self.train_dataset,
            batch_size=self.args.train_batch_size,
            shuffle=False, # IMPORTANT TO STOP OVERFITTING
#             sampler=train_sampler,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
        )

In [4]:
eurusd = Dataset.from_parquet('data/EURUSD_day.pq')

# make splits
split = eurusd.train_test_split(.1, shuffle = False)
valid_test = split['test'].train_test_split(.5, shuffle = False)
eurusd = DatasetDict({
    'train': split['train'],
    'validation': valid_test['train'],
    'test': valid_test['test']
})

Using custom data configuration default-524475b7137ed866
Found cached dataset parquet (C:/Users/micha/.cache/huggingface/datasets/parquet/default-524475b7137ed866/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [9]:
training_args = TrainingArguments(
    output_dir = "./results",
    logging_strategy = "steps",
    evaluation_strategy = "steps",
    logging_steps = 200,
    eval_steps = 200,
    report_to = "none",
    learning_rate = 5e-4,
    lr_scheduler_type = "cosine",
    warmup_ratio = .05,
    num_train_epochs = 1,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    max_grad_norm = 1,
)

PyTorch: setting up devices


In [10]:
config = GPT2Config(
    n_embd = 320, n_head = 5, vocab_size = 0, n_positions = 2000,
    resid_pdrop = .1, embd_pdrop = .1, attn_pdrop = .1,
#     resid_pdrop = .0, embd_pdrop = .0, attn_pdrop = .0,
    scale_attn_by_inverse_layer_idx = True, use_cache = False
)

In [11]:
model = GPT2Trader(config).cuda()
trainer = TraderTrainer(
    model = model,
    args = training_args,
    train_dataset = eurusd['train'],
    eval_dataset = eurusd['validation']
)

Using 13 layers


In [12]:
trainer.train()

***** Running training *****
  Num examples = 3098
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3098
Could not estimate the number of tokens of the input, floating-point operations will not be computed


sharpe 0.02251, t-value 4.239 trade % 41.04, big trade % 7.137, loss 1.784


Step,Training Loss,Validation Loss
200,0.858300,0.325155
400,0.684100,0.312105
600,0.685900,0.326843
800,0.717900,0.304278
1000,0.462200,0.298933
1200,0.448600,0.298155
1400,0.316600,0.294620
1600,0.456300,0.297838
1800,0.519900,0.295943
2000,0.388000,0.290440


sharpe 0.009464, t-value 1.774 trade % 40.69, big trade % 7.214, loss 1.429
sharpe 0.01071, t-value 2.047 trade % 42.27, big trade % 9.072, loss 1.472
sharpe 0.03608, t-value 6.811 trade % 41.25, big trade % 8.212, loss 1.316
sharpe 0.01752, t-value 3.188 trade % 38.3, big trade % 4.85, loss 1.485
sharpe 0.07246, t-value 13.34 trade % 39.24, big trade % 7.115, loss 1.134
sharpe 0.009567, t-value 1.784 trade % 40.26, big trade % 6.497, loss 1.247
sharpe 0.09239, t-value 17.87 trade % 43.28, big trade % 10.67, loss 1.029
sharpe 0.1743, t-value 33.94 trade % 43.85, big trade % 10.53, loss 1.11
sharpe 0.04428, t-value 9.19 trade % 49.85, big trade % 15.87, loss 1.211
sharpe 0.07284, t-value 15.03 trade % 49.29, big trade % 13.56, loss 1.125
sharpe 0.02509, t-value 5.57 trade % 57.03, big trade % 19.72, loss 1.302
sharpe 0.1209, t-value 26.59 trade % 56.05, big trade % 20.22, loss 1.233
sharpe 0.2662, t-value 57.11 trade % 53.27, big trade % 18.04, loss 0.9574
sharpe 0.0445, t-value 10.14 t

***** Running Evaluation *****
  Num examples = 172
  Batch size = 1


...

sharpe 0.1025, t-value 10.62 trade % 12.42, big trade % 10.79, loss 0.2922
sharpe 0.02669, t-value 3.21 trade % 16.74, big trade % 14.62, loss 0.2436
sharpe 0.1269, t-value 12.11 trade % 10.55, big trade % 8.906, loss 0.3243
sharpe 0.1132, t-value 12.96 trade % 15.19, big trade % 12.53, loss 0.2872
sharpe 0.1765, t-value 20.63 trade % 15.81, big trade % 13.41, loss 0.2339
sharpe 0.1155, t-value 13.71 trade % 16.29, big trade % 13.59, loss 0.2248
sharpe 0.151, t-value 15.55 trade % 12.27, big trade % 10.75, loss 0.3025
sharpe 0.08986, t-value 10.73 trade % 16.49, big trade % 14.22, loss 0.2663
sharpe 0.1358, t-value 13.67 trade % 11.74, big trade % 9.287, loss 0.2805
sharpe 0.1296, t-value 11.94 trade % 9.82, big trade % 3.39, loss 0.2663
sharpe 0.1059, t-value 13.16 trade % 17.88, big trade % 15.84, loss 0.2822
sharpe 0.1806, t-value 24.29 trade % 20.94, big trade % 19.56, loss 0.2317
sharpe 0.128, t-value 13.97 trade % 13.79, big trade % 10.96, loss 0.2674
sharpe 0.155, t-value 15.44 

Saving model checkpoint to ./results\checkpoint-3000
Configuration saved in ./results\checkpoint-3000\config.json
Model weights saved in ./results\checkpoint-3000\pytorch_model.bin


sharpe 0.1626, t-value 23.59 trade % 24.35, big trade % 18.2, loss 0.2934
sharpe 0.1051, t-value 8.292 trade % 7.206, big trade % 2.023, loss 0.2683
sharpe 0.03934, t-value 3.266 trade % 7.977, big trade % 2.438, loss 0.2414
sharpe 0.1072, t-value 14.58 trade % 21.41, big trade % 11.08, loss 0.2678
sharpe 0.18, t-value 32.51 trade % 37.75, big trade % 33.32, loss 0.3023
sharpe 0.06012, t-value 10.82 trade % 37.52, big trade % 32.56, loss 0.3453
sharpe 0.1592, t-value 17.21 trade % 13.51, big trade % 3.993, loss 0.3194
sharpe 0.108, t-value 19.28 trade % 36.86, big trade % 22.52, loss 0.1963
sharpe 0.117, t-value 20.87 trade % 36.82, big trade % 32.19, loss 0.3093
sharpe 0.1083, t-value 17.61 trade % 30.62, big trade % 26.03, loss 0.3637
sharpe 0.09599, t-value 14.96 trade % 28.1, big trade % 17.04, loss 0.3704
sharpe 0.1528, t-value 24.67 trade % 30.16, big trade % 23.45, loss 0.3302
sharpe 0.1406, t-value 17.57 trade % 18.07, big trade % 5.186, loss 0.324
sharpe 0.0484, t-value 3.592 



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3098, training_loss=0.4807242423661529, metrics={'train_runtime': 1148.6036, 'train_samples_per_second': 2.697, 'train_steps_per_second': 2.697, 'total_flos': 0.0, 'train_loss': 0.4807242423661529, 'epoch': 1.0})

In [ ]:
del trainer
del model
torch.cuda.empty_cache()

# Appendix

## quick timing check

In [ ]:
model = GPT2Trader(config).cuda()

In [72]:
%%timeit
fake_data = torch.randn(4, 391, 256)
fake_data = fake_data.cuda()
model(fake_data)
cpu = fake_data.cpu()

28.2 ms ± 2.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
model = GPT2Trader(config).cpu()

Using 9 layers


In [74]:
%%timeit
fake_data = torch.randn(4, 391, 256)
model(fake_data)

748 ms ± 82.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
